## fine tuning using data from retail

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
from zipfile import ZipFile
import random

In [2]:
pd.set_option('display.max_colwidth',None)

In [3]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

In [4]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch_device = 'cpu'

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1',
                           device=torch_device)
num_epochs = 5
train_batch_size = 64

2022-11-14 14:33:14 - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1


In [ ]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.80
margin = 0.90

dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/retail_data/')

model_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                               'data/output/training_retailConstrativeLoss-')\
+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


os.makedirs(model_save_path, exist_ok=True)

In [ ]:
if not os.path.exists(dataset_path):
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/tableA.csv',
                  path=os.path.join(dataset_path,'tableA.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/tableB.csv',
                  path=os.path.join(dataset_path,'tableB.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/test.csv',
                  path=os.path.join(dataset_path,'test.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/train.csv',
                  path=os.path.join(dataset_path,'train.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/valid.csv',
                  path=os.path.join(dataset_path,'valid.csv'))

In [ ]:
# Getting de data for the correct format
tableA = pd.read_csv(os.path.join(dataset_path,'tableA.csv'))
tableA['id'] = 'a_' + tableA.id.astype(str).str.zfill(4)
tableB = pd.read_csv(os.path.join(dataset_path,'tableB.csv'))
tableB['id'] = 'b_' + tableB.id.astype(str).str.zfill(4)
train = pd.read_csv(os.path.join(dataset_path,'train.csv'))
train['ltable_id'] = 'a_' + train.ltable_id.astype(str).str.zfill(4)
train['rtable_id'] = 'b_' + train.rtable_id.astype(str).str.zfill(4)
test = pd.read_csv(os.path.join(dataset_path,'test.csv'))
test['ltable_id'] = 'a_' + test.ltable_id.astype(str).str.zfill(4)
test['rtable_id'] = 'b_' + test.rtable_id.astype(str).str.zfill(4)
valid = pd.read_csv(os.path.join(dataset_path,'valid.csv'))
valid['ltable_id'] = 'a_' + valid.ltable_id.astype(str).str.zfill(4)
valid['rtable_id'] = 'b_' + valid.rtable_id.astype(str).str.zfill(4)

In [ ]:
print(len(tableA))
print(len(tableB))
print(len(train))
print(len(test))
print(len(valid))

In [ ]:
train_pairs = (train#[train.label == 1]
               .merge(tableA, left_on='ltable_id', right_on='id')
               .merge(tableB, left_on='rtable_id', right_on='id')
              )[['title_x', 'title_y', 'label']]

In [ ]:
######### Read train data  ##########
# Read train data
train_samples = []
for index, row in train_pairs.iterrows():
    sample = InputExample(texts=[row['title_x'], row['title_y']], label=int(row['label']))
    train_samples.append(sample)

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

In [ ]:
valid_pairs = (valid#[valid.label == 1]
               .merge(tableA, left_on='ltable_id', right_on='id')
               .merge(tableB, left_on='rtable_id', right_on='id')
              )[['title_x', 'title_y', 'label']]

In [ ]:
################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Questions pair classification,
# Duplicate Questions Mining, and Duplicate Questions Information Retrieval
evaluators = []

###### Classification ######
# Given (quesiton1, question2), is this a duplicate or not?
# The evaluator will compute the embeddings for both questions and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []

for index, row in valid_pairs.iterrows():
    dev_sentences1.append(row['title_x'])
    dev_sentences2.append(row['title_y'])
    dev_labels.append(int(row['label']))


binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, 
                                                                dev_sentences2, 
                                                                dev_labels)
evaluators.append(binary_acc_evaluator)

In [ ]:
###### Duplicate Mining ######
# Given a large corpus of data, identify all duplicates in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
duplicates_all = pd.concat([
    train[train.label == 1], 
    test[test.label == 1], 
    valid[valid.label == 1]]
).reset_index(drop=True)

max_dev_samples = 10000
dev_sentences = {}
dev_duplicates = []

for index, row in pd.concat([tableA, tableB], ignore_index=True).iterrows():
    dev_sentences[row['id']] = row['title']
    
    if len(dev_sentences) >= max_dev_samples:
        break
        
for index, row in duplicates_all.iterrows():
    if row['ltable_id'] in dev_sentences and row['rtable_id'] in dev_sentences:
        dev_duplicates.append([row['ltable_id'], row['rtable_id']])

In [ ]:
# The ParaphraseMiningEvaluator computes the cosine similarity between all sentences and
# extracts a list with the pairs that have the highest similarity. Given the duplicate
# information in dev_duplicates, it then computes and F1 score how well our duplicate mining worked
paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(dev_sentences, 
                                                                   dev_duplicates, name='dev')
evaluators.append(paraphrase_mining_evaluator)

In [ ]:
dev_queries = (duplicates_all.groupby('ltable_id')['rtable_id']
 .agg(','.join)
 .reset_index()
 .merge(tableA, left_on='ltable_id', right_on='id')
)[['ltable_id', 'title', 'rtable_id']]

In [ ]:
max_corpus_size = 10000

ir_queries = {}
ir_needed_qids = set()
ir_corpus = {}
ir_relevant_docs = {}

for index, row in dev_queries.iterrows():
    qid = row.ltable_id
    query = row.title
    duplicate_ids = row.rtable_id.split(',')
    ir_queries[qid] = query
    ir_relevant_docs[qid] = set(duplicate_ids)
    
    for qid in duplicate_ids:
        ir_needed_qids.add(qid)


In [ ]:
# First get all needed relevant documents (i.e., we must ensure, that the relevant questions are actually in the corpus

distraction_questions = {}
for index, row in tableB.iterrows():
    qid = row.id
    question = row.title
    if qid in ir_needed_qids:
        ir_corpus[qid] = question
    else:
        distraction_questions[qid] = question
        
# Now, also add some irrelevant questions to fill our corpus
other_qid_list = list(distraction_questions.keys())
random.shuffle(other_qid_list)

for qid in other_qid_list[0:max(0, max_corpus_size-len(ir_corpus))]:
    ir_corpus[qid] = distraction_questions[qid]

In [ ]:
#Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR
# metrices. For our use case MRR@k and Accuracy@k are relevant.
ir_evaluator = evaluation.InformationRetrievalEvaluator(ir_queries, ir_corpus, ir_relevant_docs)

evaluators.append(ir_evaluator)

In [ ]:
# Create a SequentialEvaluator. This SequentialEvaluator runs all three evaluators in a sequential order.
# We optimize the model with respect to the score from the last evaluator (scores[-1])
seq_evaluator = evaluation.SequentialEvaluator(evaluators, 
                                               main_score_function=lambda scores: scores[-1])


logger.info("Evaluate model without training")
with torch.no_grad():
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=seq_evaluator,
          epochs=num_epochs,
          warmup_steps=1000,
          output_path=model_save_path
         )